# Managed Lane Example

In [1]:
import os

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard
from network_wrangler import TransitNetwork
from network_wrangler import Scenario

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

%load_ext autoreload
%autoreload 2

### Remote I/O

In [2]:
stpaul_dir = os.path.join(os.getcwd(), '../', 'examples','stpaul')
lasso_dir = os.path.join('../../', 'lasso')

# Roadway Network files
stpaul_shape_file = os.path.join(stpaul_dir, 'shape.geojson')
stpaul_link_file = os.path.join(stpaul_dir, 'link.json')
stpaul_node_file = os.path.join(stpaul_dir, 'node.geojson')

# Project Card
roadway_project_card_file = os.path.join(
        stpaul_dir, "project_cards", '4_simple_managed_lane.yml'
    )

### Data Reads

In [3]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

transit_net = TransitNetwork.read(feed_path = stpaul_dir)

parameters = Parameters(lasso_base_dir = lasso_dir)

2020-08-18 10:56:02, INFO: Reading from following files:
-/Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/link.json
-/Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/node.geojson
-/Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/shape.geojson.
2020-08-18 10:56:07, INFO: Read 66253 links from /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/link.json
2020-08-18 10:56:07, INFO: Read 17159 nodes from /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/node.geojson
2020-08-18 10:56:07, INFO: Read 66253 shapes from /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/shape.geojson
2020-08-18 10:56:19, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul
2020-08-18 10:56:19, INFO: Removing calendar.txt from transit network config because file not found
2020-08-18 10:56:19, INFO: Removing calendar_dates.t

## Apply Project Card to Scenario

In [4]:
base_scenario = {"road_net": roadway_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2020-08-18 10:56:20, INFO: Creating Scenario


In [5]:
project_card = ProjectCard.read(roadway_project_card_file, validate = False)
working_scenario.apply_project(project_card)

2020-08-18 10:56:20, INFO: Applying test managed lane project
2020-08-18 10:56:20, INFO: Applying Project to Roadway Network: test managed lane project


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [6]:
working_scenario.applied_projects

['test managed lane project']

## Make Travel Model Network and Write to Shapefile

In [7]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(roadway_network_object = working_scenario.road_net, parameters = parameters)

2020-08-18 10:56:31, INFO: Lasso base directory set as: ../../lasso


In [8]:
m_net.roadway_standard_to_met_council_network()

2020-08-18 10:56:31, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-08-18 10:56:31, INFO: Creating managed lane network.
2020-08-18 10:56:31, INFO: Creating network with duplicated managed lanes


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


2020-08-18 10:56:32, INFO: Creating calculated roadway variables.
2020-08-18 10:56:32, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:401: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-08-18 10:56:39, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-08-18 10:56:40, INFO: Adding roadway network variable for county using a spatial join with: ../../lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:293: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-08-18 10:56:46, INFO: Finished Calculating county variable: county
2020-08-18 10:56:46, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-08-18 10:56:46, INFO: Finished calculating centroid connector variable: centroidconnect
2020-08-18 10:56:46, INFO: Calculating MPO as roadway network variable: mpo
2020-08-18 10:56:46, INFO: Finished calculating MPO variable: mpo
2020-08-18 10:56:46, INFO: Calculating Assignment Group as network variable: assign_group
2020-08-18 10:56:46, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-08-18 10:57:16, INFO: Finished calculating assignment group variable: assign_group
2020-08-18 10:57:17, INFO: Calculating Roadway Class
2020-08-18 10:57:17, INFO: Finished calculating roadway class variable: roadway_class
2020-08-18 10:57:17, INFO: Adding Counts
2020-08-18 10:57:17, INFO: Adding Variable AADT using Shared Streets Reference from ../../lasso/m

In [9]:
m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(stpaul_dir, 'links.shp'),
    output_node_shp = os.path.join(stpaul_dir, 'nodes.shp'),
    output_link_csv = os.path.join(stpaul_dir, 'links.csv'),
    output_node_csv = os.path.join(stpaul_dir, 'nodes.csv'),
    data_to_dbf = True,
)


2020-08-18 10:57:27, INFO: Writing Network as Shapefile
2020-08-18 10:57:27, INFO: Renaming DBF Node Variables
2020-08-18 10:57:27, INFO: Renaming variables so that they are DBF-safe
2020-08-18 10:57:27, INFO: Renaming DBF Link Variables
2020-08-18 10:57:27, INFO: Renaming variables so that they are DBF-safe
2020-08-18 10:57:28, INFO: Writing Node Shapes:
 - /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/nodes.shp
2020-08-18 10:57:30, INFO: Writing Link Shapes:
 - /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/links.shp
2020-08-18 10:58:41, INFO: Writing Network Data to CSVs:
 - /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/links.csv
 - /Users/wsp/Documents/GitHub/network_wrangler/notebook/../examples/stpaul/nodes.csv
